In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import time
import load_cifar
from sklearn.metrics import roc_auc_score, accuracy_score
from scipy.sparse import csr_matrix

In [2]:
tf.reset_default_graph()

In [3]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    data_shuffle = np.asarray(data_shuffle)
    labels_shuffle = np.asarray(labels_shuffle)
    labels_shuffle = np.squeeze(labels_shuffle)
    return data_shuffle, labels_shuffle

<h1>Define Hyper-perparmeter</h1>

In [15]:
num_classes = 10
num_features = 3072
num_output = 10
num_layers_0 = 500
num_layers_1 = 40
starter_learning_rate = 0.01
regularizer_rate = 0
batch_size = 2056
epochs = 100
dropout_prob = 0.8
training_accuracy = []
validation_accuracy = []
training_loss = []
testing_accuracy = []

In [5]:
#### EXAMPLE, FEEL FREE TO CHANGE #####
valid_features, valid_labels = pickle.load(open('vali_data.pickle','rb'))
training_features,training_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))

In [6]:
valid_labels = np.squeeze(valid_labels.todense())
training_labels = np.squeeze(training_labels.todense())
test_labels = np.squeeze(test_labels.todense())

In [7]:
print(training_labels.shape)
print(training_labels[0])
print(training_features.shape)

(40000, 10)
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
(40000, 3072)


In [8]:
print(training_features[0,0:100])

[0.1372549  0.10588235 0.09803922 0.10196078 0.09411765 0.08627451
 0.09019608 0.09803922 0.09803922 0.09411765 0.10196078 0.09803922
 0.30980392 0.45098039 0.44313725 0.45490196 0.45882353 0.4627451
 0.49019608 0.50980392 0.44705882 0.42745098 0.4745098  0.47843137
 0.54117647 0.49411765 0.54117647 0.55686275 0.50588235 0.51764706
 0.52156863 0.49411765 0.24705882 0.1254902  0.06666667 0.08235294
 0.09411765 0.09019608 0.09019608 0.09019608 0.09019608 0.09019608
 0.09019608 0.08627451 0.36078431 0.50980392 0.48627451 0.49803922
 0.54117647 0.54117647 0.42352941 0.4627451  0.5372549  0.5254902
 0.52941176 0.54117647 0.5372549  0.51372549 0.52941176 0.4745098
 0.43137255 0.4        0.41176471 0.39215686 0.38823529 0.19215686
 0.05882353 0.07843137 0.09411765 0.09411765 0.09019608 0.09803922
 0.10980392 0.10980392 0.09803922 0.10196078 0.39215686 0.5254902
 0.46666667 0.42745098 0.53333333 0.54117647 0.39215686 0.4627451
 0.56078431 0.54901961 0.57254902 0.60392157 0.5254902  0.55294118


<h1>Define Placeholder</h1>

In [9]:
# Placeholders for the input data
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_y = tf.placeholder('float32',shape = (None,num_classes),name="input_Y")
## for dropout layer
keep_prob = tf.placeholder(tf.float32)


In [10]:
## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_classes], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_classes]))

Instructions for updating:
Colocations handled automatically by placer.


<h1>Define Neural Network Architecture</h1>

In [11]:
## Initializing weigths and biases
hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0) + bias_0)
hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0_0,weights_1) + bias_1)
hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
predicted_y = tf.matmul(hidden_output_1_1,weights_2) + bias_2

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<h1>Define cost and optimization</h1>

In [17]:
## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_y,labels=input_y)) \
        + regularizer_rate*(tf.reduce_sum(tf.square(bias_0)) + tf.reduce_sum(tf.square(bias_1)) \
                           + tf.reduce_sum(tf.square(bias_2)))
## Variable learning rate
#learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
learning_rate = starter_learning_rate
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(input_y,1), tf.argmax(predicted_y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [18]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for epoch in range(epochs): 
        start_time = time.time()
        for index in range(0,60000,batch_size):
            batch_x, batch_y = next_batch(batch_size, training_features, training_labels)
            s.run(optimizer, feed_dict={input_X:batch_x, input_y:batch_y, keep_prob:dropout_prob})
        use_time = time.time() - start_time
        training_accuracy.append(s.run(accuracy, feed_dict= {input_X:batch_x, 
                                                             input_y:batch_y,keep_prob:1}))
        training_loss.append(s.run(loss, {input_X: batch_x, 
                                          input_y: batch_y,keep_prob:1}))

        # Evaluation of model
        validation_accuracy.append(s.run(accuracy, feed_dict= {input_X:valid_features, \
                                                               input_y:valid_labels,keep_prob:1}))
        #print(s.run(accuracy, feed_dict= {input_X:valid_features, input_y:valid_labels,keep_prob:1}))
        print("Epoch:{0}, Train time: {1:.3f} Train loss: {2:.2f} Train acc: {3:.3f}, \
        Validation acc:{4:.3f}".format(epoch, use_time, training_loss[epoch], training_accuracy[epoch], \
                              validation_accuracy[epoch]))
    test_accuracy = accuracy_score(test_labels.argmax(1), s.run(predicted_y, {input_X: test_features, keep_prob:1}).argmax(1))
    print('testing accuracy is: ', test_accuracy)

Epoch:0, Train time: 2.563 Train loss: 2.32 Train acc: 0.120,         Validation acc:0.129
Epoch:1, Train time: 2.420 Train loss: 2.10 Train acc: 0.226,         Validation acc:0.236
Epoch:2, Train time: 2.357 Train loss: 1.99 Train acc: 0.292,         Validation acc:0.267
Epoch:3, Train time: 2.369 Train loss: 1.96 Train acc: 0.283,         Validation acc:0.277
Epoch:4, Train time: 2.390 Train loss: 1.94 Train acc: 0.299,         Validation acc:0.287
Epoch:5, Train time: 2.465 Train loss: 1.93 Train acc: 0.287,         Validation acc:0.287
Epoch:6, Train time: 2.372 Train loss: 1.89 Train acc: 0.280,         Validation acc:0.283
Epoch:7, Train time: 2.412 Train loss: 1.89 Train acc: 0.287,         Validation acc:0.292
Epoch:8, Train time: 2.390 Train loss: 1.83 Train acc: 0.344,         Validation acc:0.317
Epoch:9, Train time: 2.375 Train loss: 1.84 Train acc: 0.329,         Validation acc:0.321
Epoch:10, Train time: 2.388 Train loss: 1.84 Train acc: 0.326,         Validation acc:0.31

Epoch:90, Train time: 2.378 Train loss: 1.64 Train acc: 0.412,         Validation acc:0.389
Epoch:91, Train time: 2.356 Train loss: 1.63 Train acc: 0.426,         Validation acc:0.400
Epoch:92, Train time: 2.429 Train loss: 1.63 Train acc: 0.401,         Validation acc:0.385
Epoch:93, Train time: 2.383 Train loss: 1.66 Train acc: 0.410,         Validation acc:0.402
Epoch:94, Train time: 2.472 Train loss: 1.66 Train acc: 0.429,         Validation acc:0.405
Epoch:95, Train time: 2.410 Train loss: 1.62 Train acc: 0.424,         Validation acc:0.403
Epoch:96, Train time: 2.377 Train loss: 1.65 Train acc: 0.409,         Validation acc:0.402
Epoch:97, Train time: 2.377 Train loss: 1.64 Train acc: 0.405,         Validation acc:0.409
Epoch:98, Train time: 2.404 Train loss: 1.63 Train acc: 0.433,         Validation acc:0.401
Epoch:99, Train time: 2.329 Train loss: 1.66 Train acc: 0.441,         Validation acc:0.405
testing accuracy is:  0.4263
